In [ ]:
# Perform training and testing for the budget of N_QUERIES. The "unlabeled" dataset is queried for the most informative points according to our query strategy
            # which are consecutively added to the training set after which performance is directly measured.
            for index in range(N_QUERIES):
                # Setting up kernel with rbf as default and additional parameters (for PAL and xPAL)
                params = {}
                if AL_METHOD == 6 or AL_METHOD == 7 or AL_METHOD == 8 or AL_METHOD == 9:
                    S =  pairwise_kernels(X = X_train, Y = X_train, metric='rbf', gamma=gamma)
                    S = check_array(S, np.eye(len(X_train)))
                    if SEPARATE_CLASSIFIER:
                        params = {'n_classes': len(unique_classes), "labeled_instances": query_estimator.X_training, "labels": query_estimator.y_training, "kernel": S, "priors": pal_priors}
                    else: 
                        params = {'n_classes': len(unique_classes), "labeled_instances": learner.X_training, "labels": learner.y_training, "kernel": S, "priors": pal_priors}

                # Query instance using AL method
                if SEPARATE_CLASSIFIER:
                    query_index, query_instance = query_estimator.query(X_train, **params)
                else:
                    query_index, query_instance = learner.query(X_train, **params)

                # For measuring frequencies of selection of specific trajectories over the course of learning
                trajectory_selection_frequencies.iloc[index][int(X_train[query_index,0])] += 1
                
                # For measuring performance in terms of label selection
                selected_training_instance_labels = np.append(selected_training_instance_labels, y_train[query_index])

                # logic to make sure only do additional bootstrapping when using qbc, and only on the QBC classifiers
                boot_main = False
                boot_sep = False
#                 # This currently breaks. The bootstrapping takes random samples, but there are too few in the initialization set to guarantee at least one of each class
#                 if AL_METHOD == 5:
#                     if SEPARATE_CLASSIFIER:
#                         boot_main = False
#                         boot_sep = True
#                     else:
#                         boot_main = True
#                         boot_sep = True
                
                # Setting weighting of training instances when not randomly selecting them. Using two arrays guarantees that no overlapping values erase each other
                #if WEIGHTING_FACTOR != 0 and AL_METHOD != 1 and AL_METHOD != 5:
                #    current_labels = np.append(learner.y_training, y_train[query_index].reshape(1, ))
                #    sample_weights = np.zeros(len(current_labels)).astype(float)
                #    temp_indices = np.where(current_labels == NUM_TARGET)
                #    sample_weights[temp_indices] = target_weight
                #    temp_indices = np.where(current_labels != NUM_TARGET)
                #    sample_weights[temp_indices] = non_target_weight
                #else:
                #    sample_weights = None
    
        
                # Training the consumer with the summarizing queried instance
                if TRAIN_ON_SUM:
                    # Train our ActiveLearner model with the instance it has requested.
                    X_temp, y_temp = X_train[query_index].reshape(1, -1), y_train[query_index].reshape(1, )
                    learner.teach(X=X_temp, y=y_temp, bootstrap=boot_main, sample_weight = sample_weights)

                    # Training a separate classifier (producer) with the summarizing queried instance
                    if SEPARATE_CLASSIFIER:
                        X_temp, y_temp = X_train[query_index].reshape(1, -1), y_train[query_index].reshape(1, )
                        query_estimator.teach(X=X_temp, y=y_temp, bootstrap=boot_sep, sample_weight = sample_weights)

                # Training the model with all instances of queried trip
                else:
                    # Training a separate classifier (producer) used for selecting queries
                    if SEPARATE_CLASSIFIER:
                        X_temp, y_temp = X_train[query_index].reshape(1, -1), y_train[query_index].reshape(1, )
                        
                        #if WEIGHTING_FACTOR != 0 and AL_METHOD != 1 and AL_METHOD != 5:
                        #    current_labels = np.append(query_estimator.y_training, y_train[query_index].reshape(1, ))
                        #    sample_weights = np.zeros(len(current_labels)).astype(float)
                        #    temp_indices = np.where(current_labels == NUM_TARGET)
                        #    sample_weights[temp_indices] = target_weight
                        #    temp_indices = np.where(current_labels != NUM_TARGET)
                        #    sample_weights[temp_indices] = non_target_weight
                        #else:
                        #    sample_weights = None
                        
                        query_estimator.teach(X=X_temp, y=y_temp, bootstrap = boot_sep, sample_weight = sample_weights)

                    # Find trip_id of selected query in aggregated instances
                    query_trajectory_id = X_train[query_index,0]
                    # Retrieve all instances with this trajectory id from full data set

                    if not isinstance(query_trajectory_id, float):
                        query_trajectory_id = query_trajectory_id[0]
                    query_trajectory_full = fold_full_df[fold_full_df[trajectory] == query_trajectory_id] # for sep classifier the specific formatting returns one element in two dimensions

                    # Split into numpy X and y
                    X_train_full = query_trajectory_full.drop(target, axis=1).to_numpy()
                    y_train_full = query_trajectory_full[target].to_numpy()
                    del(query_trajectory_full)

                    #if WEIGHTING_FACTOR != 0 and AL_METHOD != 1 and AL_METHOD != 5:
                    #    current_labels = np.append(learner.y_training, y_train_full)
                    #    sample_weights = np.zeros(len(current_labels)).astype(float)
                    #    temp_indices = np.where(current_labels == NUM_TARGET)
                    #    sample_weights[temp_indices] = target_weight
                    #    temp_indices = np.where(current_labels != NUM_TARGET)
                    #    sample_weights[temp_indices] = 1 - (non_target_weight * 0.1)
                    #else:
                    #    sample_weights = None
                    
                    # Add selected instances to training pool and teach the model re-train the model on the new set
                    # Train the consumer on either the full or 
                    learner.teach(X=X_train_full, y=y_train_full, bootstrap=boot_main, sample_weight = sample_weights)

                    del(X_train_full)
                    del(y_train_full)

                # Remove the queried instance from the unlabeled pool.
                X_train, y_train = np.delete(X_train, query_index, axis=0), np.delete(y_train, query_index)

In [ ]:
def run_openml_test(X, Y, k_, execs_, n_queries_, n_init_, train_sum_, sep_class_, ml_method_, ml_sep_method_, al_method_, qbc_learners_, n_qbc_learners_, weighting_factor_, save_figs_, save_ratio_, save_results_, time_point = None, bandwidth_ = 0, pal_priors = 1, sc_pca_ = True):
    # Seed to allow different AL methods to train and test with the same splits and initialization sets over the course of the same number of executions with the same settings.
    np.random.seed(0)
    # Call the set_settings method to change settings as defined by the parameters of this method.
    set_settings(k_, execs_, n_queries_, n_init_, train_sum_, sep_class_, ml_method_, ml_sep_method_, al_method_, qbc_learners_, n_qbc_learners_, weighting_factor_, save_figs_, save_ratio_, save_results_, bandwidth_, sc_pca_)
    
    # Keep track of results for each query. Used later to calculate incremental performance. +1 because the first value is the performance directly after initialization
    accuracy_results = pd.DataFrame(columns = range(N_QUERIES + 1))
    macro_f1_results = pd.DataFrame(columns = range(N_QUERIES + 1))
    recall_results = pd.DataFrame(columns = range(N_QUERIES + 1))
    precision_results = pd.DataFrame(columns = range(N_QUERIES + 1))
    auc_results = pd.DataFrame(columns = range(N_QUERIES + 1))
    
     # Perform K-fold validation for EXECUTIONS number of times. This gives more total repetitions, and thus a smoother view of learning performance
    for execs in range(EXECUTIONS):
        df = X

        print("Starting execution: ", execs+1, "/" , EXECUTIONS,  "- - - - - - - - ")

        unique_classes = Y.unique()
        #print(unique_classes)
        n_labeled_examples = df.shape[0]

        if len(unique_classes) > INITIALIZATION_SET:
            print("INCREASE SIZE OF INITIALIZATION SET TO INCLUDE AT LEAST ONE INSTANCE OF EACH CLASS")
            raise
        
        # Make random selection for initialization set based on the sampling ratio and initialization set size
        n_minority = math.ceil(int(INITIALIZATION_SET * SAMPLING_RATIO))
        n_majority = INITIALIZATION_SET - n_minority
        print(n_majority)
        #initialization_indices = np.hstack([df[df[target]==NUM_TARGET].sample(n=n_minority, replace=False).index, \
                                                 #df[df[target]!=NUM_TARGET].sample(n=n_majority, replace=False).index])
        #initialization_set = df.iloc[initialization_indices]

        # Isolate the non-initialization examples to make up the sampling pool.
        #df = df.drop(df.index[initialization_indices]).reset_index(drop=True)

        # Create separate set of initialization indices if using full trajectories for training
        if not TRAIN_ON_SUM:
            full_initialization_set = pd.DataFrame(columns = full_df.columns)

            for unique_trajectory in initialization_set[trajectory].unique():
                # Retrieve all instances with this trajectory id from full data set
                full_initialization_set = full_initialization_set.append(full_df[full_df[trajectory] == unique_trajectory])

            # Split into X and y
            X_initialize = full_initialization_set.drop(target, axis=1).reset_index(drop=True)
            y_initialize = full_initialization_set[target].reset_index(drop=True)
            del(full_initialization_set)
        else:
            # Labeled training instances.
            X_initialize = initialization_set.drop(target, axis=1).reset_index(drop=True)
            y_initialize = initialization_set[target].reset_index(drop=True)
            
        # labeled training instances.
        X_aggr_initialize = initialization_set.drop(target, axis=1).reset_index(drop=True)
        y_aggr_initialize = initialization_set[target].reset_index(drop=True)

        X_initialize[trajectory] = X_initialize[trajectory].apply(pd.to_numeric)
        y_initialize = pd.to_numeric(y_initialize)

        # Take stratified folds to make sure each fold contains enough instances of the majority class
        skf = StratifiedKFold(n_splits=K, shuffle = True)
        
        if time_point != None:
            pretime_df, ontime_df = time_specific_summarized_set(df, time_point, 30)
    
            X_aggr = ontime_df.drop(target, axis=1).to_numpy()
            y_aggr = ontime_df[target].to_numpy()
            train_set_indices = np.array(list(skf.split(X_aggr, y_aggr)))
            
        else:
            # Split aggregated instances into X and y
            X_aggr = df.drop(target, axis=1).to_numpy()
            y_aggr = df[target].to_numpy()
            train_set_indices = np.array(list(skf.split(X_aggr, y_aggr)))
            
        for i in range(K):
            X_train = X_aggr[train_set_indices[i][0]]
            y_train = y_aggr[train_set_indices[i][0]]

            X_test = X_aggr[train_set_indices[i][1]]
            y_test = y_aggr[train_set_indices[i][1]]
            
            # If the training set has fewer instances than the selected query budget, we have too few instances
            if len(X_train) < N_QUERIES:
                print("TOO FEW TRAINING INSTANCES TO TRAIN THIS FAR. REDUCE TO ", len(X_train))
                raise
                
                
            # Save class ratio of instances in training fold
            fold_training_ratio = np.count_nonzero(y_train == NUM_TARGET)/np.count_nonzero(y_train != NUM_TARGET) # MOSTLY MAKES SENSE FOR BINARY TARGET CLASS

            model = ML_switcher.get(ML_METHOD)
            
            learner = ActiveLearner(
                    estimator=deepcopy(model),
                    query_strategy=AL_switcher.get(AL_METHOD),
                    X_training=fold_X_initialize.to_numpy(), y_training=y_initialize.to_numpy()
            )

In [ ]:
run_openml_test(X=X,Y=y,k_ = 5, execs_ = 3, n_queries_ = 80, n_init_ = 10, train_sum_ = False, sep_class_ = True, \
             ml_method_ = 3, ml_sep_method_ = 3, al_method_ = 3, qbc_learners_ = [2], n_qbc_learners_ = 8, \
             weighting_factor_ = 0, save_figs_ = True, save_ratio_ = True, save_results_ = True, time_point = None)